<a href="https://colab.research.google.com/github/Qiaochu-Zhang/C-_Grades_with_Hash_Table/blob/main/mobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision
!pip install opencv-python

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold, train_test_split
import random
import matplotlib.pyplot as plt

# 检查是否有GPU可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 自定义数据集类
class NoiseDataset(Dataset):
    def __init__(self, image_folder, label):
        self.image_paths = [
            os.path.join(image_folder, img)
            for img in os.listdir(image_folder)
            if img.lower().endswith(('.jpg', '.jpeg'))
        ]
        self.image_paths = [path for path in self.image_paths if cv2.imread(path) is not None]
        self.labels = [label] * len(self.image_paths)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485], std=[0.229])
        ])

    def extract_noise(self, image_path):
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        blurred = cv2.GaussianBlur(img, (5, 5), 0)
        noise = cv2.absdiff(img, blurred)
        return Image.fromarray(noise)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        noise_image = self.extract_noise(self.image_paths[idx])
        img_tensor = self.transform(noise_image)
        label = self.labels[idx]
        return img_tensor, label

# 定义数据集路径
real_images_path = '/content/drive/MyDrive/Real'
ai_images_path = '/content/drive/MyDrive/fake'

# 加载数据集
real_dataset = NoiseDataset(real_images_path, label=0)
ai_dataset = NoiseDataset(ai_images_path, label=1)

# 合并数据集
full_dataset = real_dataset + ai_dataset

# 划分测试集（10% 用于测试）
labels = [label for _, label in full_dataset]
train_val_indices, test_indices = train_test_split(
    range(len(full_dataset)), test_size=0.1, stratify=labels, random_state=42
)

train_val_dataset = Subset(full_dataset, train_val_indices)
test_dataset = Subset(full_dataset, test_indices)

print(f"Training+Validation set size: {len(train_val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# 加载预训练的MobileNetV2模型并修改第一层卷积层
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
original_conv = model.features[0][0]

model.features[0][0] = nn.Conv2d(
    in_channels=1,
    out_channels=original_conv.out_channels,
    kernel_size=original_conv.kernel_size,
    stride=original_conv.stride,
    padding=original_conv.padding,
    bias=original_conv.bias is not None
)
with torch.no_grad():
    model.features[0][0].weight = nn.Parameter(original_conv.weight.sum(dim=1, keepdim=True))

# 修改最后的全连接层为 2 类输出
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

# 将模型移动到设备上
model = model.to(device)

# 定义 K 折交叉验证
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 用于保存最佳模型权重
best_model_wts = None
best_acc = 0.0

# 开始 K 折交叉验证
for fold, (train_ids, val_ids) in enumerate(kfold.split(train_val_indices)):
    print(f'\nFold {fold + 1}\n-------------------')

    train_loader = DataLoader(Subset(train_val_dataset, train_ids), batch_size=8, shuffle=True)
    val_loader = DataLoader(Subset(train_val_dataset, val_ids), batch_size=8, shuffle=False)

    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # 训练模型
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # 验证模型
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Fold {fold + 1} Accuracy: {accuracy:.2f}%')

    # 保存最佳模型
    if accuracy > best_acc:
        best_acc = accuracy
        best_model_wts = model.state_dict()

# 加载最佳模型权重
model.load_state_dict(best_model_wts)

# 在测试集上评估模型
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

test_accuracy = 100 * correct / total
print(f'\nTest Accuracy: {test_accuracy:.2f}%')


Using device: cuda
Training+Validation set size: 268
Test set size: 30


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 52.6MB/s]



Fold 1
-------------------
Epoch [1/5], Loss: 0.6653
Epoch [2/5], Loss: 0.5591
Epoch [3/5], Loss: 0.4459
Epoch [4/5], Loss: 0.3147
Epoch [5/5], Loss: 0.2382
Fold 1 Accuracy: 88.89%

Fold 2
-------------------
Epoch [1/5], Loss: 0.2355
Epoch [2/5], Loss: 0.1276
Epoch [3/5], Loss: 0.1003
Epoch [4/5], Loss: 0.1071
Epoch [5/5], Loss: 0.1358
Fold 2 Accuracy: 100.00%

Fold 3
-------------------
Epoch [1/5], Loss: 0.0622
Epoch [2/5], Loss: 0.0621
Epoch [3/5], Loss: 0.0429
Epoch [4/5], Loss: 0.0352
Epoch [5/5], Loss: 0.0200
Fold 3 Accuracy: 100.00%

Fold 4
-------------------
Epoch [1/5], Loss: 0.0347
Epoch [2/5], Loss: 0.0293
Epoch [3/5], Loss: 0.0750
Epoch [4/5], Loss: 0.0348
Epoch [5/5], Loss: 0.0307
Fold 4 Accuracy: 100.00%

Fold 5
-------------------
Epoch [1/5], Loss: 0.0684
Epoch [2/5], Loss: 0.0299
Epoch [3/5], Loss: 0.0184
Epoch [4/5], Loss: 0.0396
Epoch [5/5], Loss: 0.0365
Fold 5 Accuracy: 100.00%

Test Accuracy: 76.67%
